# Zad 1

dupa


# Zad 2

# Zad 3

# Zad 4

# Zagadnienie 5 - zmiana miar jakości klasyfikatora przy zmniejszaniu zbioru uczącego

# Zagadnienie 6 - Augumentacja

Augumentacja jest techniką pozyskiwania dodatkowych danych do trenowania modelu na podstawie już istniejących danych poprzez aproksymację, inter- i ekstrapolację, przekształcenie itp. Potrafi znacznie poprawić dokładność modelu w sytuacji, gdy zbiór danych nie jest wystarczająco duży.

Dla przykładu prosta augumentacja obrazu może polegać na dodaniu do zbioru treningowego dodatkowych kopii tego obrazu uzyskiwanych przez przekształcenia takie jak: obrót, przybliżenie/oddalenie, przycięcie, dodanie szumu.

### Prosty przykład procedury przekształcającej obraz

In [27]:
# Procedura obracająca obraz wzdłóż poziomej osi symetrii (obraz przechowywany jako dwuwymiarowa tablica numpy)
import numpy as np
def flip(image):
    height, width = image.shape
    flipped_image = np.ndarray(shape=(height, width), dtype=int)
    for row in range(height):
        flipped_image[row] = image[height - row - 1]
    return flipped_image

In [28]:
image = np.array([[1,2,3,4,3,2],
                  [5,6,7,8,7,6],
                  [9,10,11,12,11,10],
                  [13,14,15,16,15,14]])

print("Oryginalny obraz")
print(image)
print("Odwrócony obraz")
print(flip(image))

Oryginalny obraz
[[ 1  2  3  4  3  2]
 [ 5  6  7  8  7  6]
 [ 9 10 11 12 11 10]
 [13 14 15 16 15 14]]
Odwrócony obraz
[[13 14 15 16 15 14]
 [ 9 10 11 12 11 10]
 [ 5  6  7  8  7  6]
 [ 1  2  3  4  3  2]]


Można by stwierdzić, że nikt nie będzie próbował dokonywać klasyfikowacji odwróconego zdjęcia, np. człowieka.<br>
Jednak czy aby na pewno jest ono zupełnie nieprzydatne do celu zwiększenia dokładności modelu?

![title](stanie-na-rekach.jpg)

### Jak bardzo augumentacja poprawia model?

Spróbujemy skorzystać z niewielkiego zbioru danych, dodać augumentację i sprawdzić czy dokładność modelu poprawiła się.

In [84]:
import imgaug.augmenters as iaa
# Czy rzecz tak prosta jak dodanie kilku losowych czarno-białych pikseli może pomóc zwiększyć dokładność?
def augument(images):
    aug = iaa.Sequential([
        iaa.SaltAndPepper(0.1)
    ])
    return aug(images=images)

In [85]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Użyjemy tylko 700 z 70000 obrazków w MNIST
mnist_small, mnist_small_target = mnist_frame[:700], mnist_target[:700]
X_train, X_test, y_train, y_test = train_test_split(mnist_small, mnist_small_target, train_size=0.75, random_state=0)
X_train_aug, y_train_aug = np.reshape(np.copy(X_train), (-1, 28, 28)), np.concatenate([y_train, y_train])

# Dla każdego obrazka z X_train dokładamy nowy, zaugumentowany
for aug_img in augument([np.reshape(x, (28,28,1)) for x in np.array(X_train, dtype='uint8')]):
    X_train_aug = np.append(X_train_aug, np.reshape(aug_img, (1,28,28)), axis=0)

X_train_aug = np.reshape(X_train_aug, (-1, 784))
    
params = {
  'n_neighbors': (2, 3, 5),
  'weights': ('uniform', 'distance')
}
knn = KNeighborsClassifier(algorithm='auto')
clf = GridSearchCV(knn, params)

(525, 784) (1050, 784)


In [89]:
# Sprawdzenie dokładności bez augumentacji
clf.fit(X_train, y_train)
y_pred_best = clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred_best)

0.8457142857142858

In [88]:
# Sprawdzenie dokładności po augumentacji
clf.fit(X_train_aug, y_train_aug)
y_pred_best = clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred_best)

0.8628571428571429

Dodanie do modelu nowych obrazów z szumem "salt and pepper" pomogło poprawić model<br>
Widzimy że nawet bardzo prosta augumentacja potrafi zwiększyć dokładność modelu

# Zad 7

# Zad 8